In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from django.contrib.gis.geos import Point, LineString as DjangoLineString
import pytz

In [2]:
from simple_history.utils import bulk_create_with_history
from helpers.histories import bulk_update_with_history

In [3]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [4]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [5]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )
#     print(road_set)

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
def get_connected_reference(company, resource, key, value):
    value=value.strip()
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [8]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)
    if field == 'Nome da placa':
        value=value.upper()
    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [9]:
def get_sign(field, sign_field, occ, value):
    if sign_field == 'Cor':
        value=value.capitalize()
    elif sign_field == 'Tipo de Película':
        value=value.split(' ')[1]
    values = occ.form_fields['fields']
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [10]:
!cat .env

STAGE=CCR_PRODUCTION


In [11]:
# filename = 'acessos_importacao_v2'
filename='SH MV BR-116 SP RJ - Inventário Corrigido' # 16/05/23

In [12]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[0]
# sheetname
ws = wb[sheetname]
ws

<Worksheet "MV - BR-116 SP RJ">

In [13]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    if obj['Rodovia'] is not None:           
        obj['img'] = [a for a in ws._images if a.anchor._from.row == index]
 
    values.append(obj)

In [14]:
for a in values:
    if a['Cor_1'] == 'Branca':
        a['Cor_1'] = "Branco"
    if a['Cor_2'] == 'Branca':
        a['Cor_2'] = "Branco"
    if a['Cor_3'] == 'Branca':
        a['Cor_3'] = "Branco"
    if a['Cor_4'] == 'Branca':
        a['Cor_4'] = "Branco"
    if a['Cor_5'] == 'Branca':
        a['Cor_5'] = "Branco"
    if a['Cor_6'] == 'Branca':
        a['Cor_6'] = "Branco"
    if a['Cor_7'] == 'Branca':
        a['Cor_7'] = "Branco"
    if a['Cor_8'] == 'Branca':
        a['Cor_8'] = "Branco"
    if a['Cor_9'] == 'Branca':
        a['Cor_9'] = "Branco"
    if a['Cor_10'] == 'Branca':
        a['Cor_10'] = "Branco"
    if a['Cor_1'] == 'Amarela':
        a['Cor_1'] = "Amarelo"
    if a['Cor_2'] == 'Amarela':
        a['Cor_2'] = "Amarelo"
    if a['Cor_3'] == 'Amarela':
        a['Cor_3'] = "Amarelo"
    if a['Cor_4'] == 'Amarela':
        a['Cor_4'] = "Amarelo"
    if a['Cor_5'] == 'Amarela':
        a['Cor_5'] = "Amarelo"
    if a['Cor_6'] == 'Amarela':
        a['Cor_6'] = "Amarelo"
    if a['Cor_7'] == 'Amarela':
        a['Cor_7'] = "Amarelo"
    if a['Cor_8'] == 'Amarela':
        a['Cor_8'] = "Amarelo"
    if a['Cor_9'] == 'Amarela':
        a['Cor_9'] = "Amarelo"
    if a['Cor_10'] == 'Amarela':
        a['Cor_10'] = "Amarelo"

In [15]:
values[0]

{'Identificação ANTT': ' MV 116 RJ 214+840 N 1',
 'km': ' 214+840',
 'Status': 'Identificado',
 'Equipe': 'Civil: SGS',
 'Encontrado em': datetime.datetime(2022, 11, 4, 0, 0),
 'Executado em': datetime.datetime(2022, 11, 4, 0, 0),
 'Sentido': 'Norte',
 'Classe': 'Sinalização Horizontal (Marcas Viárias)',
 'Faixa': 'Expressa',
 'Rodovia': 'BR-116 RJ',
 'Localização': 'Expressa',
 'Observações': None,
 'Latitude': -22.71358,
 'Longitude': -43.73373,
 'Latitude (UTM)': 630048.95,
 'Longitude (UTM)': 7487631.32,
 'Zona (UTM)': '23K',
 'Tipo de Marca Viária_1': 'Seta',
 'Cor_1': 'Branco',
 'Descrição_1': 'Seta',
 'Tipo de Marca Viária_2': None,
 'Cor_2': None,
 'Descrição_2': None,
 'Tipo de Marca Viária_3': None,
 'Cor_3': None,
 'Descrição_3': None,
 'Tipo de Marca Viária_4': None,
 'Cor_4': None,
 'Descrição_4': None,
 'Tipo de Marca Viária_5': None,
 'Cor_5': None,
 'Descrição_5': None,
 'Tipo de Marca Viária_6': None,
 'Cor_6': None,
 'Descrição_6': None,
 'Tipo de Marca Viária_7': Non

In [16]:
company = Company.objects.get(name='CCR - Rio Sp')

occurrence_type=OccurrenceType.objects.get(uuid='29c1bb9b-16b0-4e7d-9b78-2221447b3d6b')
user=User.objects.get(username='rlcs')
# utc = pytz.timezone('America/Sao_Paulo')
company,occurrence_type

(<Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>,
 <OccurrenceType: Sinalização Horizontal (Marcas Viárias) - ['CCR - Rio Sp']>)

In [17]:
for a in tqdm(values):
    a['Cores'] = list(filter(None, [{'color': get_sign("Adicionar retrorrefletância de longitudinal","Cor",occurrence_type,a["Cor_1"]), 'type': a['Tipo de Marca Viária_1'],'description':a['Descrição_1'],'length': a['Comprimento (m)_1'] if a.get('Comprimento (m)_1') else None,"width": a["Largura (m)_1"] if a.get('Largura (m)_1') else None,'area': a['Area (m²)_1'] if a.get('Area (m²)_1') else None} if a['Cor_1'] is not None else None,
                                    {'color': get_sign("Adicionar retrorrefletância de longitudinal","Cor",occurrence_type,a["Cor_2"]), 'type': a['Tipo de Marca Viária_2'],'description':a['Descrição_2'],'length': a['Comprimento (m)_2'] if a.get('Comprimento (m)_2') else None,"width": a["Largura (m)_2"] if a.get('Largura (m)_2') else None,'area': a['Area (m²)_2'] if a.get('Area (m²)_2') else None} if a['Cor_2'] is not None else None,
                                    {'color': get_sign("Adicionar retrorrefletância de longitudinal","Cor",occurrence_type,a["Cor_3"]), 'type': a['Tipo de Marca Viária_3'],'description':a['Descrição_3'],'length': a['Comprimento (m)_3'] if a.get('Comprimento (m)_3') else None,"width": a["Largura (m)_3"] if a.get('Largura (m)_3') else None,'area': a['Area (m²)_3'] if a.get('Area (m²)_3') else None} if a['Cor_3'] is not None else None,
                                    {'color': get_sign("Adicionar retrorrefletância de longitudinal","Cor",occurrence_type,a["Cor_4"]), 'type': a['Tipo de Marca Viária_4'],'description':a['Descrição_4'],'length': a['Comprimento (m)_4'] if a.get('Comprimento (m)_4') else None,"width": a["Largura (m)_4"] if a.get('Largura (m)_4') else None,'area': a['Area (m²)_4'] if a.get('Area (m²)_4') else None} if a['Cor_4'] is not None else None,
                                    {'color': get_sign("Adicionar retrorrefletância de longitudinal","Cor",occurrence_type,a["Cor_5"]), 'type': a['Tipo de Marca Viária_5'],'description':a['Descrição_5'],'length': a['Comprimento (m)_5'] if a.get('Comprimento (m)_5') else None,"width": a["Largura (m)_5"] if a.get('Largura (m)_5') else None,'area': a['Area (m²)_5'] if a.get('Area (m²)_5') else None} if a.get('Cor_5') is not None else None,
                                    {'color': get_sign("Adicionar retrorrefletância de longitudinal","Cor",occurrence_type,a["Cor_6"]), 'type': a['Tipo de Marca Viária_6'],'description':a['Descrição_6'],'length': a['Comprimento (m)_6'] if a.get('Comprimento (m)_6') else None,"width": a["Largura (m)_6"] if a.get('Largura (m)_6') else None,'area': a['Area (m²)_6'] if a.get('Area (m²)_6') else None} if a.get('Cor_6') is not None else None,
                                    {'color': get_sign("Adicionar retrorrefletância de longitudinal","Cor",occurrence_type,a["Cor_7"]), 'type': a['Tipo de Marca Viária_7'],'description':a['Descrição_7'],'length': a['Comprimento (m)_7'] if a.get('Comprimento (m)_7') else None,"width": a["Largura (m)_7"] if a.get('Largura (m)_7') else None,'area': a['Area (m²)_7'] if a.get('Area (m²)_7') else None} if a.get('Cor_7') is not None else None,
                                    {'color': get_sign("Adicionar retrorrefletância de longitudinal","Cor",occurrence_type,a["Cor_8"]), 'type': a['Tipo de Marca Viária_8'],'description':a['Descrição_8'],'length': a['Comprimento (m)_8'] if a.get('Comprimento (m)_8') else None,"width": a["Largura (m)_8"] if a.get('Largura (m)_8') else None,'area': a['Area (m²)_8'] if a.get('Area (m²)_8') else None} if a.get('Cor_8') is not None else None,
                                    {'color': get_sign("Adicionar retrorrefletância de longitudinal","Cor",occurrence_type,a["Cor_9"]), 'type': a['Tipo de Marca Viária_9'],'description':a['Descrição_9'],'length': a['Comprimento (m)_9'] if a.get('Comprimento (m)_9') else None,"width": a["Largura (m)_9"] if a.get('Largura (m)_9') else None,'area': a['Area (m²)_9'] if a.get('Area (m²)_9') else None} if a.get('Cor_9') is not None else None,
                                    {'color': get_sign("Adicionar retrorrefletância de longitudinal","Cor",occurrence_type,a["Cor_10"]), 'type': a['Tipo de Marca Viária_10'],'description':a['Descrição_10'],'length': a['Comprimento (m)_10'] if a.get('Comprimento (m)_10') else None,"width": a["Largura (m)_10"] if a.get('Largura (m)_10') else None,'area': a['Area (m²)_10'] if a.get('Area (m²)_10') else None} if a.get('Cor_10') is not None else None]))

  0%|          | 0/445 [00:00<?, ?it/s]

In [25]:
objects=[]

for index, a in enumerate(tqdm(values)):

    point, road = get_road_coordinates(a['Rodovia'], float(str(a['km']).replace('+','.')), str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize())) , company)
    if road.name != a['Rodovia']:
        print(road.name,a['Rodovia'])
        
    try:
        objects.append(Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    firm= Firm.objects.get(company=company, name=a['Equipe']),
                    lane=get_connected_reference(company, 'reporting', 'lane', a['Faixa']),
                    road=road,
                    road_name=road.name,
                    direction=get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize()),
                    created_by=user,
                    found_at=a['Encontrado em'],
                    executed_at=a['Executado em'],
                    km=float(str(a['km']).replace('+','.')),
                    end_km_manually_specified= True if 'km final' in a else False,
                    end_km=float(a['km final']) if 'km final' in a and a['km final'] is not None else None,
                    point=point,
                    form_data={
                        "localization":a['Localização'] if a['Localização'] is not None else None,
                        "id_ccr_antt":a['Identificação ANTT'].strip(),
                        "index":a['Índice (Serial Antt)'] if 'Índice (Serial Antt)' in a else None,
                        "retro":a['Cores'] if a['Cores'] is not None else None,
                        "notes":a['Observações'] if a.get('Observações') is not None else '',
                        "latitude":a['Latitude (UTM)'] if a['Latitude (UTM)'] is not None else '',
                        "longitude":a['Longitude (UTM)'] if a['Longitude (UTM)'] is not None else '',
                        "zone":get_value('Zona (UTM)',occurrence_type,a['Zona (UTM)']) if a['Zona (UTM)'] is not None else None,
                        "latitudedecimal":str(a['Latitude']) if a.get('Latitude') else '',
                        "longitudedecimal":str(a['Longitude']) if a.get('Longitude') else '',
                        },
                    ))
    except Exception as e:
        print(f'AQUI: km: {a["km"]} | Rodovia: {a["Rodovia"]} | Linha da Planilha: {index+2}{e}{objecr.form}')
        pass

  0%|          | 0/445 [00:00<?, ?it/s]

In [26]:
objects[10].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f1131a24e50>,
 'uuid': UUID('a2f059db-7f7a-428c-b3bb-35a07b282eba'),
 'number': '',
 'company_id': UUID('3af64f25-59e8-446b-bb1e-963549090b0d'),
 'road_name': 'BR-116 RJ',
 'road_id': 2037,
 'km': 233.2,
 'end_km': None,
 'km_reference': None,
 'project_km': 0,
 'project_end_km': None,
 'point': <Point object at 0x7f1120178760>,
 'direction': '1',
 'lane': '29',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': UUID('8e3f277d-cd02-444f-a49b-c8d724f92c9d'),
 'occurrence_type_id': UUID('29c1bb9b-16b0-4e7d-9b78-2221447b3d6b'),
 'form_data': {'localization': 'Expressa',
  'id_ccr_antt': 'MV 116 RJ 233+200 S 1',
  'index': None,
  'retro': [{'color': '2',
    'type': 'Seta',
    'description': 'Seta',
    'length': None,
    'width': None,
    'area': None}],
  'notes': '',
  'latitude': 619108.58,
  'longitude': 7493060.71,
  'zone': '1',
  'latitudedecimal': '-22.6

In [27]:
for reporting in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/445 [00:00<?, ?it/s]

'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...


/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2022-11-04 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2022-11-04 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2022-11-04 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: Da

Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.76407
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.76408
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.76409
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.76410
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.76411
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.76412
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ


/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2022-11-11 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2022-11-11 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2022-11-11 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: Da

...

CCR-Inv-2023.76662
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76663
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76664
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76665
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76666
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76667
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76668
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76669
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1: BR-116 S

CCR-Inv-2023.76724
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76725
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76726
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP


/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2022-11-14 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2022-11-14 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2022-11-14 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: Da

CCR-Inv-2023.76727
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76728
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76729
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76730
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76731
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76732
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76733
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on R

CCR-Inv-2023.76787
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76788
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76789
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76790
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76791
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76792
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76793
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.76794
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1: BR-116 S

In [43]:
# for reporting, images in tqdm(objects):
#     for image in reversed(images):
#         reporting_file = ReportingFile(
#             created_by=user,
#             reporting=reporting,
#             description=image.name,
#             km=reporting.km,
#             point=reporting.point
#         )
#         reporting_file.save()
#         image_io = BytesIO(image._data())
#         im = Image.open(image_io)
#         if im.mode in ("RGBA", "P"):
#             im = im.convert("RGB")
#         thumb_io = BytesIO()
#         im.save(thumb_io, format='jpeg', quality=90)
#         reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/585 [00:00<?, ?it/s]

In [20]:
reps=Reporting.objects.filter(company=company,occurrence_type=occurrence_type,road_name__startswith='BR-116')
reps.count()

0

In [19]:
reps.delete()

(445, {'reportings.Reporting': 445})

In [ ]:
def get_connected_reference(company, resource, key, value):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [ ]:
teste = get_connected_reference(company, 'reporting', 'lane', value)

In [22]:
for a in tqdm(values):
    rep=Reporting.objects.get(company=company,occurrence_type=occurrence_type,
                              form_data__id_ccr_antt=a['Identificação ANTT'],
                              form_data__latitude=a['Latitude (UTM)'],
                              form_data__longitude=a['Longitude (UTM)'])
    rep.form_data['latitudedecimal'] = a['Latitude ']
    rep.form_data['longitudedecimal'] = a['Longitude ']
    rep.save()

  0%|          | 0/5 [00:00<?, ?it/s]

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
